In [1]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
# Load the data into a DataFrame
data = pd.read_csv(filepath_or_buffer='final_data_only_tpm.csv')

# Split the data into features (X) and target variable (y)
X = data.drop('cancer_type', axis=1)
y = data['cancer_type']

# Split the data into training and validation sets
train_dataset, test_dataset = train_test_split(data, test_size=0.2, random_state=42)

data.head()

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,AC092910.4,AC073611.1,AC136977.1,AC078856.1,AC008763.4,AL592295.6,AC006486.3,AL391628.1,AP006621.6,cancer_type
0,19.7011,0.1125,46.1397,1.8913,0.2670,18.6466,4.3945,24.1591,6.7242,9.4371,...,0.0,0.0641,0.0,1.0888,0.0,3.1977,0.0,0.0104,0.3953,kidney chromophobe
1,31.6275,0.0000,45.5189,2.8743,0.4744,4.2348,1.4145,60.8153,15.4309,7.5267,...,0.0,0.0781,0.0,0.0000,0.0,15.4262,0.0,0.0042,0.4094,kidney chromophobe
2,35.9928,0.5207,108.0270,4.8134,0.8010,4.5446,3.2850,33.1630,13.1085,12.1888,...,0.0,0.1049,0.0,0.0000,0.0,13.2312,0.0,0.1154,0.3050,kidney chromophobe
3,18.2605,0.0921,27.5398,2.3258,0.3473,4.5023,1.5078,47.2006,10.6721,5.0657,...,0.0,0.1193,0.0,0.0000,0.0,6.4516,0.0,0.0141,0.1416,kidney chromophobe
4,68.8052,3.4134,88.0619,6.2415,1.2251,9.5310,30.6990,66.3841,12.9370,20.1289,...,0.0,0.1141,0.0,0.0000,0.0,21.8857,0.0,0.0394,0.3547,kidney chromophobe


In [2]:
# Split the data into training and validation sets
import torch
# Split the data into features (X) and target variable (y)


map = {
    "kidney chromophobe": 0,
    "kidney renal papillary cell carcinoma": 1,
    "kidney renal clear cell carcinoma": 2,
}

data1 = data.copy()
data1['cancer_type'] = data1['cancer_type'].replace(map)

X = data1.drop('cancer_type', axis=1)
y = data1['cancer_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tens = torch.tensor(X_train.values).float()
X_test_tens = torch.tensor(X_test.values).float()
y_train_tens = torch.tensor(y_train.values)
y_test_tens = torch.tensor(y_test.values)

data1.head()

y_test_tens

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2,
        2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 0, 0, 2, 1, 2, 1, 2, 2, 2, 0,
        1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 0, 2, 1, 1, 2, 1, 2, 1, 1, 2, 0, 2, 2,
        0, 2, 1, 0, 0, 2, 2, 2, 0, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2,
        2, 1, 0, 1, 1, 2, 2, 1, 0, 1, 0, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1,
        1, 2, 2, 0, 0, 1, 2, 1, 2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2,
        2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 1, 2, 1,
        2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2])

In [3]:
import torch.utils.data
import torch.nn as nn
from torch.utils.data import DataLoader

batch_size = 128

train_dataloader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)
val_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size)

# Build model
class MultiClassificationModel(nn.Module):
    def __init__(self, input_features, output_features, hidden_units=8):
        """Initializes all required hyperparameters for a multi-class classification model.

        Args:
            input_features (int): Number of input features to the model.
            out_features (int): Number of output features of the model
              (how many classes there are).
            hidden_units (int): Number of hidden units between layers, default 8.
        """
        super().__init__()

        """ n_input_channels = 3  # i.e., length of our OHE scheme
        n_filters = 32  # i.e., no. of kernels
        kernel_size = 3
        conv_out_len = output_features - kernel_size + 1
        self.net = nn.Sequential(
            nn.Conv1d(
                in_channels=n_input_channels,
                out_channels=n_filters,
                kernel_size=kernel_size,
            ),
            nn.ReLU(inplace=True),
            nn.Flatten(),
            nn.Linear(in_features=n_filters * conv_out_len, out_features=1),
        ) # sizes don't add up.. "Given groups=1, weight of size [32, 3, 3], expected input[1, 822, 60660] to have 3 channels, but got 822 channels instead" """


        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.ReLU(), # <- does our dataset require non-linear layers? (try uncommenting and see if the results change)
            #nn.Sigmoid(),
            #nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(), # <- does our dataset require non-linear layers? (try uncommenting and see if the results change)
            nn.Linear(in_features=hidden_units, out_features=output_features), # how many classes are there?
            #nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        return self.linear_layer_stack(x)

# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [4]:
NUM_FEATURES = 60660
NUM_CLASSES = 3
device = "cpu"

# Create an instance of BlobModel and send it to the target device
model = MultiClassificationModel(input_features=NUM_FEATURES, 
                    output_features=NUM_CLASSES, 
                    hidden_units=8).to(device)
model

MultiClassificationModel(
  (linear_layer_stack): Sequential(
    (0): Linear(in_features=60660, out_features=8, bias=True)
    (1): ReLU()
    (2): ReLU()
    (3): Linear(in_features=8, out_features=3, bias=True)
  )
)

In [5]:
LR = 0.1

# Create loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=LR) # exercise: try changing the learning rate here and seeing what happens to the model

In [6]:
# Perform a single forward pass on the data (we'll need to put it to the target device for it to work)
model(X_train_tens.to(device))[:5]

tensor([[ 17.9070,   8.3835, -21.0792],
        [ 63.4400,  27.8139, -60.5553],
        [ 52.0934,  18.6517, -22.2563],
        [  4.5515,  29.3419,  22.4233],
        [ 24.5839,  12.0705, -32.3873]], grad_fn=<SliceBackward0>)

In [7]:
model(X_train_tens.to(device))[0].shape, NUM_CLASSES 

(torch.Size([3]), 3)

In [8]:
# Make prediction logits with model
y_logits = model(X_test_tens.to(device))

# Perform softmax calculation on logits across dimension 1 to get prediction probabilities
y_pred_probs = torch.softmax(y_logits, dim=1) 
print(y_logits[:5])
print(y_pred_probs[:5])

tensor([[ 56.2720,  15.6873,   5.3981],
        [ 40.5763,  19.7952, -52.1831],
        [ 82.9150,  38.4061, -92.4706],
        [ 22.0901,  10.8266, -29.0411],
        [ 43.9705,  21.1270, -54.3569]], grad_fn=<SliceBackward0>)
tensor([[1.0000e+00, 2.3676e-18, 8.0491e-23],
        [1.0000e+00, 9.4386e-10, 5.1896e-41],
        [1.0000e+00, 4.6778e-20, 0.0000e+00],
        [9.9999e-01, 1.2833e-05, 6.2227e-23],
        [1.0000e+00, 1.2000e-10, 1.9758e-43]], grad_fn=<SliceBackward0>)


In [9]:
# Sum the first sample output of the softmax activation function 
torch.sum(y_pred_probs[0])

tensor(1., grad_fn=<SumBackward0>)

In [10]:
# Which class does the model think is *most* likely at the index 0 sample?
print(y_pred_probs[0])
print(torch.argmax(y_pred_probs[0]))

tensor([1.0000e+00, 2.3676e-18, 8.0491e-23], grad_fn=<SelectBackward0>)
tensor(0)


In [11]:
# Fit the model
#torch.manual_seed(42)

# Set number of epochs
epochs = 100

# Put data to target device
X_train_tens, y_train_tens = X_train_tens.to(device), y_train_tens.to(device)
X_test_tens, y_test_tens = X_test_tens.to(device), y_test_tens.to(device)

for epoch in range(epochs):
    ### Training
    model.train()

    # 1. Forward pass
    y_logits = model(X_train_tens) # model outputs raw logits 
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # go from logits -> prediction probabilities -> prediction labels
    # print(y_logits)
    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train_tens) 
    acc = accuracy_fn(y_true=y_train_tens,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model(X_test_tens)
      test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
      # 2. Calculate test loss and accuracy
      test_loss = loss_fn(test_logits, y_test_tens)
      test_acc = accuracy_fn(y_true=y_test_tens,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%") 

Epoch: 0 | Loss: 68.09641, Acc: 14.48% | Test Loss: 29623990.00000, Test Acc: 28.16%


Epoch: 10 | Loss: 0.96062, Acc: 59.25% | Test Loss: 0.95124, Test Acc: 61.65%
Epoch: 20 | Loss: 0.93257, Acc: 59.25% | Test Loss: 0.92527, Test Acc: 61.65%
Epoch: 30 | Loss: 0.91651, Acc: 59.25% | Test Loss: 0.91084, Test Acc: 61.65%
Epoch: 40 | Loss: 0.90671, Acc: 59.25% | Test Loss: 0.90249, Test Acc: 61.65%
Epoch: 50 | Loss: 0.90040, Acc: 59.25% | Test Loss: 0.89749, Test Acc: 61.65%
Epoch: 60 | Loss: 0.89615, Acc: 59.25% | Test Loss: 0.89444, Test Acc: 61.65%
Epoch: 70 | Loss: 0.89319, Acc: 59.25% | Test Loss: 0.89258, Test Acc: 61.65%
Epoch: 80 | Loss: 0.89108, Acc: 59.25% | Test Loss: 0.89145, Test Acc: 61.65%
Epoch: 90 | Loss: 0.88955, Acc: 59.25% | Test Loss: 0.89079, Test Acc: 61.65%
